In [5]:
from swat import *
import swat as sw
from pprint import pprint
%matplotlib inline
import matplotlib
import sys

sys.path.append('/cas/DeepLearn/weshiz/github_workspace/python-dlpy/')
sys.path.append('/cas/DeepLearn/weshiz/github_workspace/python-fcmp')
from dlpy.layers import * 
from dlpy.applications import *
from dlpy import Model, Sequential
from dlpy.utils import *
from dlpy.splitting import two_way_split
from dlpy.images import *
from dlpy.model import *
from python_fcmp.parser import *
from python_fcmp.decorator import *
from python_fcmp import fcmp
from dlpy.lr_scheduler import *
from dlpy.network import *

import numpy
import pandas as pd
import pdb

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
s = sw.CAS('dlgrd009', 23309)
s.loadactionset('deeplearn')
s.loadactionset('clustering')

NOTE: Added action set 'deeplearn'.
NOTE: Added action set 'clustering'.


[actionset]

 'clustering'

+ Elapsed: 0.00221s, user: 0.00109s, sys: 0.00105s, mem: 0.225mb

In [8]:
# https://go.documentation.sas.com/?docsetId=lesysoptsref&docsetTarget=n16rhscxem9ljwn1kuhn5170xkbg.htm&docsetVersion=9.4&locale=en
s.sessionProp.setSessOpt(caslib='CASUSER', cmplib="CASUSER.fcmpfunction", cmpopt="ALL")

NOTE: 'CASUSER(weshiz)' is now the active caslib.


+ Elapsed: 0.000214s, user: 0.0002s, mem: 0.256mb

In [9]:
@out_args('y_out')  # pass by reference
@cast_array('srcY', 'weights', 'y_out')  # declare the arguments as array type
def forward_prop(srcHeight, srcWidth, srcDepth, srcY, weights, y_out):
    o_width = 8
    o_height = 8
    o_channel = 3
    
    i_hw = srcHeight * srcWidth
    
    s_h = (srcHeight-1)/(o_height-1)
    s_w = (srcWidth-1)/(o_width-1)
    
    for m in range(o_channel):
        for j in range(o_height):
            for i in range(o_width):
                in_y = s_h * j
                in_x = s_w * i
                top_y = fcmp.floor(in_y)
                bottom_y = fcmp.ceil(in_y)
                y_lerp = in_y - top_y
                
                left_x = fcmp.floor(in_x)
                right_x = fcmp.ceil(in_x)
                x_lerp = in_x - left_x
                
                top_y_bound = max(min(top_y, srcHeight-1), 0)
                bottom_y_bound = max(min(bottom_y, srcHeight-1), 0)
                
                left_x_bound = max(min(left_x, srcWidth-1), 0)
                right_x_bound = max(min(right_x, srcWidth-1), 0)

                
                # get four interpolated points
                top_left = srcY[m*i_hw+top_y_bound*srcWidth+left_x_bound]
                top_right = srcY[m*i_hw+top_y_bound*srcWidth+right_x_bound]
                bottom_left = srcY[m*i_hw+bottom_y_bound*srcWidth+left_x_bound]
                bottom_right= srcY[m*i_hw+bottom_y_bound*srcWidth+right_x_bound]
                
                top = top_left*(1 - x_lerp) + top_right*x_lerp
                bottom = bottom_left * (1 - x_lerp) + bottom_right * x_lerp
                
                y_out[m*o_height*o_width+j*o_width+i] = top * (1 - y_lerp) + bottom * y_lerp
    
    return 

@out_args('gradient_out', 'srcDeltas_out')
@cast_array('srcY', 'Y', 'weights', 'deltas', 'gradient_out', 'srcDeltas_out')
def back_prop(srcHeight, srcWidth, srcDepth, srcY, Y, weights, deltas, gradient_out, srcDeltas_out):
    for i in range(srcHeight*srcWidth*srcDepth):
        srcDeltas_out[i] = 0
    
    return

def dummy_loss(t, target):
    return t

Arguments, srcY, weights, y_out, are casted to array type
Arguments, y_out, are declared as outargs.
Arguments, srcY, Y, weights, deltas, gradient_out, srcDeltas_out, are casted to array type
Arguments, gradient_out, srcDeltas_out, are declared as outargs.


In [10]:
forward_fcmp_code = python_to_fcmp(func=forward_prop, print=True)

('function forward_prop(srcHeight, srcWidth, srcDepth, srcY[*], weights[*], y_out[*]);outargs y_out;\n'
 '    o_width = 8;\n'
 '    o_height = 8;\n'
 '    o_channel = 3;\n'
 '    i_hw = (srcHeight * srcWidth);\n'
 '    s_h = ((srcHeight - 1) / (o_height - 1));\n'
 '    s_w = ((srcWidth - 1) / (o_width - 1));\n'
 '    do m = 0 to o_channel - 1 by 1;\n'
 '        do j = 0 to o_height - 1 by 1;\n'
 '            do i = 0 to o_width - 1 by 1;\n'
 '                in_y = (s_h * j);\n'
 '                in_x = (s_w * i);\n'
 '                top_y = floor(in_y);\n'
 '                bottom_y = ceil(in_y);\n'
 '                y_lerp = (in_y - top_y);\n'
 '                left_x = floor(in_x);\n'
 '                right_x = ceil(in_x);\n'
 '                x_lerp = (in_x - left_x);\n'
 '                top_y_bound = max(min(top_y, (srcHeight - 1)), 0);\n'
 '                bottom_y_bound = max(min(bottom_y, (srcHeight - 1)), 0);\n'
 '                left_x_bound = max(min(left_x, (srcWidth - 1

In [11]:
A = np.random.randint(0, 5, 4*4*3)

In [12]:
A.reshape(4, 4, 3)

array([[[0, 0, 1],
        [3, 0, 0],
        [0, 1, 4],
        [4, 1, 1]],

       [[2, 1, 0],
        [1, 4, 0],
        [1, 3, 4],
        [3, 2, 2]],

       [[2, 2, 1],
        [3, 2, 1],
        [4, 0, 3],
        [1, 1, 4]],

       [[1, 2, 0],
        [2, 4, 1],
        [1, 3, 2],
        [1, 0, 4]]])

In [13]:
y = np.zeros((8 * 8 * 3))
forward_prop(4, 4, 3, A, None, y)

In [14]:
y.reshape(8, 8, 3)

array([[[0.        , 0.        , 0.        ],
        [0.28571429, 0.71428571, 1.28571429],
        [2.14285714, 3.        , 0.        ],
        [0.        , 0.        , 0.16326531],
        [0.40816327, 0.79591837, 1.46938776],
        [2.14285714, 0.        , 0.        ],
        [0.        , 0.04081633, 0.10204082],
        [0.30612245, 0.79591837, 1.28571429]],

       [[1.14285714, 1.14285714, 1.14285714],
        [0.89795918, 0.53061224, 0.3877551 ],
        [0.69387755, 1.        , 2.85714286],
        [2.85714286, 2.85714286, 2.24489796],
        [1.32653061, 0.75510204, 0.87755102],
        [1.        , 3.71428571, 3.65306122],
        [3.59183673, 2.79591837, 1.63265306],
        [0.87755102, 0.93877551, 1.        ]],

       [[2.85714286, 2.6122449 , 2.36734694],
        [1.75510204, 0.95918367, 0.51020408],
        [0.75510204, 1.        , 2.        ],
        [1.57142857, 1.14285714, 0.71428571],
        [0.28571429, 0.14285714, 0.57142857],
        [1.        , 4.       

In [15]:
backward_fcmp_code = python_to_fcmp(func=back_prop, print=True)

('function back_prop(srcHeight, srcWidth, srcDepth, srcY[*], Y[*], weights[*], deltas[*], gradient_out[*], srcDeltas_out[*]);outargs gradient_out, srcDeltas_out;\n'
 '    do i = 0 to ((srcHeight * srcWidth) * srcDepth) - 1 by 1;\n'
 '        srcDeltas_out[i + 1] = 0;\n'
 '    end;\n'
 '    return ;\n'
 'endsub;\n')


In [16]:
# register forward and backward function together
register_fcmp_routines(s,
                       routine_code= forward_fcmp_code + backward_fcmp_code,
                       function_tbl_name='fcmpfunction')

NOTE: Cloud Analytic Services saved the file FCMPFUNCTION.sashdat in caslib CASUSER(weshiz).


In [17]:
test = ImageTable.load_files(conn=s, path='/cas/DeepLearn/weshiz/data')

In [18]:
test.resize(4, 4)

In [19]:
test.summarizeimages()

NOTE: Table IMAGEDATA_RI2OGK contains compressed images.
NOTE: 179 out of 179 images were scanned and results were saved to the summary table.


,Column,jpg,minWidth,maxWidth,minHeight,maxHeight,meanWidth,meanHeight,mean1stChannel,min1stChannel,max1stChannel,mean2ndChannel,min2ndChannel,max2ndChannel,mean3rdChannel,min3rdChannel,max3rdChannel
0,_image_,179.0,4.0,4.0,4.0,4.0,4.0,4.0,66.390363,0.0,255.0,71.651885,0.0,255.0,71.80831,0.0,255.0


In [20]:
test.freq(inputs='_label_')

,Column,CharVar,FmtVar,Level,Frequency
0,_label_,chemical_test,chemical_test,1,7.0
1,_label_,face,face,2,2.0
2,_label_,mask,mask,3,85.0
3,_label_,sas_football,sas_football,4,85.0


In [21]:
model = Sequential(conn=s, model_table='lenet5')

model.add(InputLayer(n_channels=3, width=4, height=4, scale=1.0/255, offsets=None,
                     random_flip=None, random_crop=None))

# instance normalizationi
model.add(FCMPLayer(width=8, height=8, depth=3, n_weights=0, 
                    forward_func='forward_prop', backward_func='back_prop',
                    name='FCMPLayer1'))

model.add(OutputLayer(n=4))

NOTE: Input layer added.
NOTE: FCMP layer added.
NOTE: Output layer added.
NOTE: Model compiled successfully.


In [22]:
model.print_summary()

,Layer Id,Layer,Type,Kernel Size,Stride,Activation,Output Size,Number of Parameters,FLOPS(forward pass)
0,0,Input1,input,,,None,"(4, 4, 3)","(0, 0)",0
1,1,FCMPLayer1,FCMP,,,None,"(8, 8, 3)","(0, 0)",0
2,2,Output1,output,,,Softmax,4,"(768, 4)",0
3,,,,,,,,Total number of parameters,Total FLOPS
4,Summary,,,,,,,772,0


In [23]:
solver = VanillaSolver(clip_grad_max=100, clip_grad_min=-100, learning_rate=0.001,
                       learning_rate_policy='step', gamma=0.1, step_size=30)
optimizer = Optimizer(algorithm=solver, seed=13309, max_epochs=1, log_level=5, mini_batch_size=32, reg_l2=0.0001)

In [24]:
model.fit(data=test, optimizer=optimizer, n_threads=8, target='_label_', inputs='_image_', nominals='_label_')

NOTE: Training from scratch.
NOTE: Due to data distribution, miniBatchSize has been limited to 30.
NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 772.
NOTE:  The approximate memory cost is 1.00 MB.
NOTE:  Loading weights cost       0.00 (s).
NOTE:  Initializing each layer cost       0.01 (s).
NOTE:  The total number of threads on each worker is 8.
NOTE:  The total mini-batch size per thread on each worker is 32.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 256.
NOTE:  Target variable: _label_
NOTE:  Number of levels for the target variable:      4
NOTE:  Levels for the target variable:
NOTE:  Level      0: chemical_test
NOTE:  Level      1: face         
NOTE:  Level      2: mask         
NOTE:  Level      3: sas_football 
NOTE:  Number of input variables:     1
NOTE:  Number of numeric input variables:      1
NOTE:  Number of FCMP layers in model: 1
NOTE:  FCMP layer 'fcmplayer1' has input tensor size: width=4, height=

,Descr,Value
0,Model Name,lenet5
1,Model Type,Convolutional Neural Network
2,Number of Layers,3
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,0
6,Number of Pooling Layers,0
7,Number of Fully Connected Layers,0
8,Number of FCMP Layers,1
9,Number of Weight Parameters,768


In [25]:
model.predict(test, layer_image_type='wide', layers=['FCMPLayer1', 'Input1'], layer_out='out')

NOTE: Due to data distribution, miniBatchSize has been limited to 3.


[ScoreInfo]

                          Descr         Value
 0  Number of Observations Read           179
 1  Number of Observations Used           179
 2  Misclassification Error (%)      86.59218
 3                   Loss Error      1.619212

[OutputCasTables]

             casLib              Name  Rows  Columns  \
 0  CASUSER(weshiz)               out   179      244   
 1  CASUSER(weshiz)  Valid_Res_2HKkgS   179       11   
 
                                             casTable  
 0          CASTable('out', caslib='CASUSER(weshiz)')  
 1  CASTable('Valid_Res_2HKkgS', caslib='CASUSER(w...  

+ Elapsed: 0.045s, user: 0.0626s, sys: 0.0418s, mem: 337mb

In [26]:
s.altertable('out', drop='_image_')

+ Elapsed: 0.000203s, user: 0.000142s, sys: 4.1e-05s, mem: 0.282mb

In [27]:
s.fetch('out').Fetch.loc[0]

_filename_0           Aflatoxin B1.PNG
_label_                  chemical_test
_id_                                 1
_LayerAct_0_0_0_0_                   1
_LayerAct_0_0_0_1_            0.992157
                            ...       
_LayerAct_1_2_7_3_            0.994398
_LayerAct_1_2_7_4_            0.997759
_LayerAct_1_2_7_5_             0.99944
_LayerAct_1_2_7_6_            0.997759
_LayerAct_1_2_7_7_            0.996078
Name: 0, Length: 243, dtype: object

In [28]:
layer_out = s.fetch('out').Fetch.iloc[0, 3:].values

In [29]:
layer_out[:4*4*3].reshape(4, 4, 3)

array([[[1.0, 0.9921569228172302, 1.0],
        [1.0, 1.0, 0.9843137860298157],
        [1.0, 0.9803922176361084, 1.0],
        [0.5490196347236633, 0.9843137860298157, 1.0]],

       [[1.0, 0.9921569228172302, 1.0],
        [0.9960784912109375, 1.0, 0.9921569228172302],
        [1.0, 1.0, 1.0],
        [0.9843137860298157, 1.0, 0.9803922176361084]],

       [[1.0, 0.5490196347236633, 0.9843137860298157],
        [1.0, 1.0, 0.9921569228172302],
        [1.0, 0.9960784912109375, 1.0],
        [0.9921569228172302, 1.0, 1.0]],

       [[1.0, 0.9843137860298157, 1.0],
        [0.9803922176361084, 1.0, 0.5490196347236633],
        [0.9843137860298157, 1.0, 1.0],
        [0.9921569228172302, 1.0, 0.9960784912109375]]], dtype=object)

In [30]:
layer_out[4*4*3:].reshape(8, 8, 3)

array([[[1.0, 0.9966386556625366, 0.993277370929718],
        [0.9943978190422058, 0.9977591037750244, 1.0],
        [1.0, 1.0, 1.0],
        [0.995198130607605, 0.9903962016105652, 0.9919968247413635],
        [0.9967987537384033, 0.9987995028495789, 0.9951980710029602],
        [0.9915966391563416, 1.0, 0.9937575459480286],
        [0.9875150322914124, 0.989595890045166, 0.9958383440971375],
        [0.9975990653038025, 0.9903962016105652, 0.9831933379173279]],

       [[1.0, 0.9399760365486145, 0.8799520134925842],
        [0.8986794948577881, 0.9567827582359314, 0.9941576719284058],
        [0.9900760650634766, 0.9859944581985474, 1.0],
        [0.8600240349769592, 0.7200480699539185, 0.7635054588317871],
        [0.8986794948577881, 0.989595890045166, 0.9919968247413635],
        [0.994397759437561, 1.0, 0.833853542804718],
        [0.667707085609436, 0.7192477583885193, 0.8796318769454956],
        [0.9883953928947449, 0.9939175844192505, 0.9994397759437561]],

       [[1.0, 0.91

In [31]:
s.endsession()

+ Elapsed: 6.7e-05s, sys: 6.3e-05s, mem: 0.203mb